## Import librairies

In [22]:
import torch
import numpy as np 
import pandas as pd
from tqdm import tqdm, trange
import torch
import pickle
import time

from hparams import hparams
hparams.parse_argument(args=["--batch_size", "128"])
 

-------------------------
Hyper Parameter Settings
-------------------------
batch_size: 128
num_epochs: 100
learning_rate: 0.1
stopping_rate: 1e-05
momentum: 0.6
factor: 0.2
patience: 30
condition: improvement
input_size: 1
hidden_size: 64
output_size: 64
pool_size: 3
threshold: 0.5
checkpoint_interval: 10
-------------------------


In [3]:
model_MLP_vgg = torch.load('./models_vggish_MLP/model_MLP_4.pth')
model_MLP_clap = torch.load('./model_best_clap_sounds/model_MLP_2.pth')
model_MLP_pann = torch.load('./models_pann-wavegram_MLP/model_MLP_8.pth')
model_MLP_pann32k = torch.load('./models_pann-32k_MLP/model_MLP_5.pth')

In [5]:
with open('./DeepFake_Real_Sounds.pkl', 'rb') as file:
    data_final = pickle.load(file)
data_final

,class,embedding,embedding_type,label,system_id,track,path_file,id
0,dog_bark,"[[-0.6465, -0.705, -0.2708, -1.329, -0.2389, -...",vggish,1,TASys02,A,./DCASE_2023/DCASE_2023_Challenge_Task_7_Submi...,0
1,dog_bark,"[[-0.564, -0.25, 0.7427, -0.3833, 0.01987, -0....",vggish,1,TASys02,A,./DCASE_2023/DCASE_2023_Challenge_Task_7_Submi...,1
2,dog_bark,"[[-0.33, -0.2524, -0.2025, -1.947, -0.776, -1....",vggish,1,TASys02,A,./DCASE_2023/DCASE_2023_Challenge_Task_7_Submi...,2
3,dog_bark,"[[-0.7124, -0.6235, -0.579, -1.739, -0.384, -1...",vggish,1,TASys02,A,./DCASE_2023/DCASE_2023_Challenge_Task_7_Submi...,3
4,dog_bark,"[[-0.2844, -0.4236, 0.2023, -1.006, -0.4763, -...",vggish,1,TASys02,A,./DCASE_2023/DCASE_2023_Challenge_Task_7_Submi...,4
...,...,...,...,...,...,...,...,...
22195,sneeze_cough,"[[0.0, 1.358, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",panns-cnn14-32k,0,None,None,./DCASE_2023/DCASE_2023_Challenge_Task_7_Datas...,122995
22196,sneeze_cough,"[[0.0, 0.652, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",panns-cnn14-32k,0,None,None,./DCASE_2023/DCASE_2023_Challenge_Task_7_Datas...,122996
22197,sneeze_cough,"[[0.0, 1.222, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",panns-cnn14-32k,0,None,None,./DCASE_2023/DCASE_2023_Challenge_Task_7_Datas...,122997
22198,sneeze_cough,"[[0.0, 1.58, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004833...",panns-cnn14-32k,0,None,None,./DCASE_2023/DCASE_2023_Challenge_Task_7_Datas...,122998


In [17]:
data_final[data_final['path_file'].str.contains('TB')]['path_file'].values




array(['./DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions\\B\\TBSys01\\dog_bark\\embeddings/vggish\\00.npy',
       './DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions\\B\\TBSys01\\dog_bark\\embeddings/vggish\\01.npy',
       './DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions\\B\\TBSys01\\dog_bark\\embeddings/vggish\\02.npy',
       ...,
       './DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions\\B\\TBSys31\\sneeze_cough\\embeddings/panns-cnn14-32k\\97.npy',
       './DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions\\B\\TBSys31\\sneeze_cough\\embeddings/panns-cnn14-32k\\98.npy',
       './DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions\\B\\TBSys31\\sneeze_cough\\embeddings/panns-cnn14-32k\\99.npy'],
      dtype=object)

In [20]:
list_models = {'vggish': model_MLP_vgg, 'clap-2023': model_MLP_clap, 'panns-cnn14-32k': model_MLP_pann32k, 'panns-wavegram-logmel': model_MLP_pann}
runs = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time_dict = {}  # dictionary for inference times in (s)
times_list = {}  # dictionary for the list of times, useful for std dev


for embedding in list_models.keys():
    model = list_models[embedding]
    model.eval()
    model.to(device)
    times = []
    audio_sample =np.load(f'./DCASE_2023/DCASE_2023_Challenge_Task_7_Submission/AudioFiles/Submissions/B/TBSys07/footstep/embeddings/{embedding}/94.npy')
    data_tensor = torch.tensor(audio_sample, dtype=torch.float)
    data = torch.mean(data_tensor, dim=0).view(1, -1)
    data = data.to(device) 
    for run in trange(runs):

        start_time = time.time()  # Start time
        with torch.no_grad():
            output = model(data)

        end_time = time.time()  # End time
        elapsed_time = (end_time - start_time) 
        times.append(elapsed_time)

    time_dict[embedding] = (np.mean(times), np.std(times))
    times_list[embedding] = times

print(time_dict)

100%|██████████| 1000/1000 [00:00<00:00, 3694.15it/s]

{'vggish': (0.0002786426544189453, 0.0004499579394031478), 'clap-2023': (0.00027860093116760255, 0.0004498855879888764), 'panns-cnn14-32k': (0.0002733004093170166, 0.000448979318218973), 'panns-wavegram-logmel': (0.00026471281051635744, 0.0004451611074151875)}
